In [ ]:
# %pip install accelerate
# %pip install -i https://pypi.org/simple/bitsandbytes
# %pip install quanto

In [ ]:
# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import torch
from transformers import AutoModelForCausalLM, QuantoConfig

import gc
gc.collect()
torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
    "thenlper/gte-small",
    device_map="cuda:0",
    torch_dtype=torch.float16,
    quantization_config=QuantoConfig(weights="int8"),
)

In [ ]:
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small")
torch.cuda.empty_cache()


In [ ]:

instruction = "Describe a futuristic device that revolutionizes space travel."


PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


In [ ]:
text = (
    PROMPT_DICT["prompt_no_input"].format(instruction=instruction)
    if not input
    else PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
)

model_inputs = tokenizer(text, return_tensors="pt").to("cuda")
output_ids = model.generate(
    **model_inputs,
    max_length=256,
)
output_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
# clean_output = output_text.split("### Response:")[1].strip()
output_text 

from IPython.display import display,Markdown
display(Markdown(output_text))